In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

Load the data in and limit the number of data in each simulation if we want to 

In [2]:
data = np.load("dataFormatted/Train.npy", allow_pickle=True)
target = np.load("dataFormatted/TrainTarget.npy", allow_pickle=True)
totalSims = 100

newData = []
newTarget = []

for dataLine, targetLine in zip(data, target):
    if targetLine <= 120:
        newData.append(dataLine)
        newTarget.append(targetLine)
        
data = np.array(newData)
target = np.array(newTarget)

Perform normalisation and drop rows which don't change

In [3]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
df = pd.DataFrame(data)
df = df.drop([0, 9, 17, 18], axis=1)

normTrain = scaler.fit_transform(df.to_numpy())

Also perform standardisation and pca if we want to use that instead

In [4]:
scaler = preprocessing.StandardScaler()

scaledTrain = scaler.fit_transform(data)

from sklearn.decomposition import PCA
pca = PCA(.96)
pcaTrain = pca.fit_transform(scaledTrain)


Create objects for all the model types that we want to test

In [5]:
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor

In [6]:
sgd = linear_model.SGDRegressor(loss='squared_loss')


forest = ensemble.RandomForestRegressor(max_depth=20, n_estimators=100)
extraForest = ensemble.ExtraTreesRegressor(n_estimators=20, max_depth=100)

svr = SVR(kernel='poly', coef0=4)

linearRgr = LinearRegression()
knn = KNeighborsRegressor(75, weights='distance')


Shuffle the data and specify the metrics we want to use

In [7]:
from sklearn.utils import shuffle

normTrain, target = shuffle(normTrain, target)
scores = {"mae" : "neg_mean_absolute_error", "mse" : "neg_mean_squared_error"}

Do a 5-fold cross validation on all the models we want to try and get their results

In [8]:
cross_validate(forest, normTrain, target, scoring = scores)

{'fit_time': array([13.06249547, 12.89413929, 12.86224532, 12.8781538 , 12.83642244]),
 'score_time': array([0.09596801, 0.09497046, 0.09598732, 0.09898472, 0.09598351]),
 'test_mae': array([-29.3923459 , -29.43481285, -29.07253647, -29.62705085,
        -29.84146957]),
 'test_mse': array([-1734.42696026, -1741.19607608, -1687.35171479, -1768.0966751 ,
        -1755.70468386])}

In [9]:
cross_validate(extraForest, normTrain, target, scoring = scores)

{'fit_time': array([1.29568577, 1.28725791, 1.27862287, 1.2959857 , 1.28017616]),
 'score_time': array([0.02499676, 0.02499294, 0.02700806, 0.02599216, 0.02600217]),
 'test_mae': array([-29.79015023, -30.23635482, -29.42978672, -30.08502181,
        -30.14678866]),
 'test_mse': array([-1790.3860292 , -1833.71656568, -1740.63278841, -1829.26853248,
        -1768.91272964])}

In [10]:
cross_validate(svr, normTrain, target, scoring = scores)

{'fit_time': array([9.64917564, 9.53342819, 9.69411087, 9.59468079, 9.66517615]),
 'score_time': array([1.09727979, 1.09868741, 1.09601712, 1.11081266, 1.10028124]),
 'test_mae': array([-30.13400307, -30.09208724, -29.25715856, -29.97358274,
        -30.09215236]),
 'test_mse': array([-1837.39688881, -1843.54219679, -1752.69495152, -1853.82687282,
        -1819.61734879])}

In [11]:
cross_validate(linearRgr, normTrain, target, scoring = scores)

{'fit_time': array([0.02199221, 0.01199412, 0.00899673, 0.00999594, 0.00999689]),
 'score_time': array([0.00100088, 0.00100017, 0.00100088, 0.00100088, 0.00099874]),
 'test_mae': array([-34.32219693, -34.2233644 , -33.70595577, -34.25344997,
        -34.12978461]),
 'test_mse': array([-2019.2214763 , -1998.99086514, -1951.68248376, -2033.283318  ,
        -1986.18359625])}

In [12]:
cross_validate(knn, normTrain, target, scoring = scores)

{'fit_time': array([0.17994165, 0.20195818, 0.21694469, 0.17255902, 0.17794394]),
 'score_time': array([1.93001127, 2.01618719, 2.25633359, 2.02799082, 2.175915  ]),
 'test_mae': array([-30.83391901, -30.58364671, -30.00065154, -30.46534143,
        -30.66016327]),
 'test_mse': array([-1827.71221643, -1806.84785153, -1745.28904987, -1808.00794861,
        -1789.49252936])}

In [13]:
cross_validate(sgd, normTrain, target, scoring = scores)

{'fit_time': array([0.13795614, 0.12495995, 0.12695909, 0.09297037, 0.09896779]),
 'score_time': array([0.00099969, 0.00100064, 0.00099945, 0.00099921, 0.00099993]),
 'test_mae': array([-34.20780039, -34.18553588, -33.80312103, -34.48812959,
        -33.93317873]),
 'test_mse': array([-2027.49004665, -2018.53989868, -1955.29336586, -2043.48835898,
        -1995.57979531])}